# Métodos Computacionles para las Políticas Públicas
# Proyecto Final - Análisis Envolvente de Datos
## Santiago Ortiz Ortiz

Importamos las librerias necesarias para desarrollar Web Scraping a la página.

In [10]:
import pandas as pd
import re
import requests
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains

**Para ingresar al panel principal.**

La página web a la que le vamos a hacer WS es Sistema de Gestión Hospitalaria (SIHO).
Url principal: https://prestadores.minsalud.gov.co/siho/
Está es la URL principal. Sin embargo, cuando uno quiere examinar la página se da cuenta que tiene unas macro que no se muestran de forma tan simple. Por lo tanto, ingresamos a un link secundario que habilita la macro: https://prestadores.minsalud.gov.co/siho/work.aspx

Una vez acá, el software nos habilita un usuario de invitado. Este ya está digitado, solo hay que darle click en ingresar.

In [11]:
# Creamos una variable con la URL de la página.
url = "https://prestadores.minsalud.gov.co/siho/work.aspx"

# Con ayuda del WebDriver de Chrome vamos a abrir la página.
browser = webdriver.Chrome(executable_path=r"C:\Users\tatoo\Desktop\PhantomJS\Chrome\chromedriver.exe")
# A la hora de hacer Web Scraping es importante dejar un periodo de tiempo relativamente largo (algo que aparente ser humano)
time.sleep(5)
# Con el .get(url), el programa sabe que tiene que ingresar a esa URL
browser.get(url)
time.sleep(5)
# Le pedimos al programa que encuentre un elemento en el código HTML que se llame "btnIngresar" y que le dé CLICK
boton_submit = browser.find_element_by_name("btnIngresar").click()
time.sleep(4)

Vamos a crear dos listas, una con los años y otra con los departamentos que tendremos que analizar.

In [ ]:
años_ops = ["2017","2018"]
deptos_ops = ["Cundinamarca", "Guainía", "Guaviare"]

In [12]:
años_ops = ["2002", "2003", "2004", "2005", "2006", "2007",
            "2008", "2009", "2010", "2011", "2012", "2013",
            "2014", "2015", "2016", "2017", "2018"] # Desde el 2002 hasta el 2018
deptos_ops = ["Amazonas", "Antioquia", "Arauca", "Atlántico", "Archipiélago de San Andrés, Providencia y Santa Catalina",
             "Bogotá, D.C.", "Bolívar", "Boyacá", "Caldas", "Caquetá", "Casanare", "Cauca", "Cesar",
             "Chocó", "Córdoba", "Cundinamarca", "Guainía", "Guaviare", "Huila",
             "La Guajira", "Magdalena", "Meta", "Nariño", "Norte de Santander",
             "Putumayo", "Quindío", "Risaralda", "Santander", "Sucre",
             "Tolima", "Valle del Cauca", "Vaupés", "Vichada"] # 32 Departamentos y el Distrito Capital

**Para que Python sepa que esta en otra pestaña**
https://stackoverflow.com/questions/17676036/python-webdriver-to-handle-pop-up-browser-windows-which-is-not-an-alert

# Distribución Recurso Humano

In [13]:
# Fijamos una página principal llamada MAIN
main_window_handle = None
while not main_window_handle:
    main_window_handle = browser.current_window_handle
browser.find_element_by_link_text('Distribución Recurso Humano').click()
signin_window_handle = None
while not signin_window_handle:
    for handle in browser.window_handles:
        if handle != main_window_handle:
            signin_window_handle = handle
            break
browser.switch_to.window(signin_window_handle)

In [ ]:
browser.page_source # Para comprobar que tenga la nueva forma

Lo que hacen los dos códigos siguientes es: 
- Encontrar las casillas año y departamento. Como ambas son elementos de multiple opción este desplegará unos valores.
- Año: 2002 - 2019 (Sin embargo, no hay datos para el 2019)
- Depto: Amazonas - Vaupés (+Bogotá DC.)

# Creación base del dataframe

In [5]:
df_base = pd.DataFrame(columns=['Nivel', 'Concepto', 'Cantidad', 'Departamento', 'Año', 'Clase'])
df_base

,Nivel,Concepto,Cantidad,Departamento,Año,Clase


# Función para extraer los datos de las páginas

In [14]:
def extract_cap(años, departamento, df_base):
    # El espacio del año tiene un nombre como el siguiente:
    cod_año = browser.find_element_by_name("_ctl0:ContentPlaceHolder1:ddano")
    for option in cod_año.find_elements_by_tag_name('option'):
        if option.text == años:
            option.click() 
            break
    time.sleep(1)
    # El espacio del departamento tiene un nombre como el siguiente:
    cod_depto = browser.find_element_by_name("_ctl0:ContentPlaceHolder1:dddepa_codigo")
    for option in cod_depto.find_elements_by_tag_name('option'):
        if option.text == departamento:
            option.click() # select() in earlier versions of webdriver
            break
    time.sleep(2)
    # Después de seleccionar los departamentos y los años a Buscar
    boton_buscar = browser.find_element_by_name("_ctl0:ibBuscarFtr")
    boton_buscar.click()
    time.sleep(3)
    html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")   
    soup = BeautifulSoup(html, "html.parser")
    time.sleep(2)
    agregar = re.findall('<td align="center">(\d*)</td><td>(.*)</td><td align="right">([\d.,]*)</td><td align="right">[\d,.]*</td><td align="right">[\d,.]*</td>', str(soup))
    if len(agregar) == 0:
        pass
    time.sleep(3)
    agregar_base = pd.DataFrame(agregar, columns = ['Nivel', 'Concepto', 'Cantidad']) 
    time.sleep(1)
    depto = []
    for i in range(len(agregar_base["Nivel"])):
        depto.append(departamento)
    agregar_base["Departamento"] = depto
    anno = []
    for i in range(len(agregar_base["Nivel"])):
        anno.append(años)
    agregar_base["Año"] = anno
    clase = []
    for i in range(len(agregar_base["Nivel"])):
        clase.append("Distribución Recurso Humano")
    agregar_base["Clase"] = clase
    df_base = df_base.append(agregar_base, ignore_index=True, sort=False)
    return df_base

In [15]:
for años in años_ops:
    for departamento in deptos_ops:
        df_base = extract_cap(años, departamento, df_base)

In [16]:
df_base

,Nivel,Concepto,Cantidad,Departamento,Año,Clase
0,1,ADMINISTRATIVO,61,Amazonas,2003,Distribución Recurso Humano
1,2,ADMINISTRATIVO,40,Amazonas,2003,Distribución Recurso Humano
2,1,ASISTENCIAL,149,Amazonas,2003,Distribución Recurso Humano
3,2,ASISTENCIAL,52,Amazonas,2003,Distribución Recurso Humano
4,3,ADMINISTRATIVO,361,Antioquia,2003,Distribución Recurso Humano
...,...,...,...,...,...,...
4826,3,OPERATIVO,361,Valle del Cauca,2018,Distribución Recurso Humano
4827,2,APOYO,20,Vaupés,2018,Distribución Recurso Humano
4828,2,OPERATIVO,20,Vaupés,2018,Distribución Recurso Humano
4829,2,APOYO,19,Vichada,2018,Distribución Recurso Humano


In [17]:
df_base.to_excel("Distribucion_rec_hum.xlsx", index = False)

**Ideas para la base de datos:**
- https://thispointer.com/python-pandas-how-to-add-rows-in-a-dataframe-using-dataframe-append-loc-iloc/

Hasta acá funciona melo!, pero hay que pensar en una forma de hacer que Python sepa que va a trabajar en la otra ventana
- Revisar: https://stackoverflow.com/questions/32858434/switch-to-popup-in-python-using-selenium